**OLSQ DAC'22 tutorial**

It is recommended to pull from [our GitHub repo](https://github.com/UCLA-VAST/OLSQ) becasue it is sometimes more up-to-date, and there are more branches containing research progress.
However, we upload OLSQ to Python Package Index (`pip`) as well, which we use here.

In [2]:
!pip install OLSQ

zsh:1: command not found: pip


The name of the package is `OLSQ`, the name of the module is `olsq`, and the name of the main solver class is `OLSQ`.
Additionally, we need to import a class `qcdevice` to load to hardware information.

In [ ]:
from olsq import OLSQ 
from olsq.device import qcdevice

We initiate the solver with an **objective** and a **mode**.
The objective can be `depth`, `swap`, or `fidelity`.
The mode can be `normal` or `transition`.

In [ ]:
solver = OLSQ("depth", "normal")

We construct a `qcdevice` object containing hardware information.
The arguments are: name, number of qubits `nqubits`, list of edges between the qubits `connection`, and duration of a SWAP gate `swap_duration`.
In our case, the QPU has five qubits connected in a 'bowtie' pattern.
We assume a SWAP gate is decomposed to three CNOTs.

<img src="./bowtie.png" width="300">

In [ ]:
bowtie_connections = [(0,1), (0,2), (1,2), (2,3), (2,4), (3,4)]
bowtie_dev = qcdevice("bowtie", nqubits=5, connection=bowtie_connections, 
                  swap_duration=3)

solver.setdevice(bowtie_dev)

We specify the program to compile with the intermediate representation of OLSQ consisting of three objects: number of physical qubits, a list of qubits involved in each gate `gate_qubits`, and a list of spec of these gates `gate_name`.
In `gate_qubits`, every element is a tuple: if the gate is on one qubit, the tuple has only one elements, otherwise the tuple has two elements. `gate_name` keeps all the other information and is not involved in the solving process.

In [ ]:
gate_qubits = [(0,), (1,), (3,), (2,3), (0,), (1,), (2,), (3,), (0,1), (2,3),
               (3,0), (1,2), (0,1), (2,3), (0,), (1,), (2,), (3,), (0,1), (2,3),
               (3,), (3,0), (3,)]

gate_name = ["x", "x", "h", "cnot", "t", "t", "t", "tdg", "cnot", "cnot", 
             "cnot", "cnot", "cnot", "cnot", "tdg", "tdg", "tdg", "t", "cnot",
             "cnot", "s", "cnot", "h"]

solver.setprogram([4, gate_qubits, gate_name], input_mode="IR")

The circuit diagram of this quantum program is shown below.
We can compare with the output in the following cell.

<img src="./adder.png" width="750">

In [ ]:
# print out the OLSQ IR
for i in range(len(gate_qubits)):
    if len(gate_qubits[i]) == 1:
        print(f"g{i} {gate_name[i]} q{gate_qubits[i][0]}")
    else:
        print(f"g{i} {gate_name[i]} q{gate_qubits[i][0]} q{gate_qubits[i][1]}")

Solve the problem with `solve()` method.
It will try to increase the depth limit if no solution is found.
The return value contains the five objects below.
There will also be some results printed out.

In [ ]:
result_depth, list_scheduled_gate_name, list_scheduled_gate_qubits,\
final_mapping, objective_value = solver.solve(output_mode="IR")

In [ ]:
print(f"objective value: {objective_value}")
print("program:")
for t in range(result_depth):
    print(f"time {t}:")
    for i in range(len(list_scheduled_gate_name[t])):
        if len(list_scheduled_gate_qubits[t][i]) == 1:
            print(f"    {list_scheduled_gate_name[t][i]} " +
                  f"p{list_scheduled_gate_qubits[t][0]}")
        else:
            print(f"    {list_scheduled_gate_name[t][i]} " +
                  f"p{list_scheduled_gate_qubits[t][0]} " +
                  f"p{list_scheduled_gate_qubits[t][1]} ")
print("final_mapping")
for i, j in enumerate(final_mapping):
    print(f"q{i} -> p{j}")

In [ ]:
# setting the objective to swap and try again
solver_swap = OLSQ("swap", "normal")
solver_swap.setdevice(bowtie_dev)
solver_swap.setprogram([4, gate_qubits, gate_name], input_mode="IR")
results_swap = solver_swap.solve(output_mode="IR")

In [ ]:
# use the transition-based model, should run much faster
solver_transition = OLSQ("swap", "transition")
solver_transition.setdevice(bowtie_dev)
solver_transition.setprogram([4, gate_qubits, gate_name], input_mode="IR")
results_transition = solver_transition.solve(output_mode="IR")